## Hands-on Tutorial: Quantifying and Reducing Gender Stereotypes in Word Embeddings

Ensuring fairness in algorithmically-driven decision-making is important to avoid inadvertent cases of bias and perpetuation of harmful stereotypes. However, modern natural language processing techniques, which learn model parameters based on data, might rely on implicit biases presented in the data to make undesirable stereotypical associations. Such a danger is facing us with word embedding, a popular framework to represent text data as vectors which has been used in many machine learning and natural language processing tasks. Recent results ([1](https://arxiv.org/abs/1607.06520), [2](https://arxiv.org/abs/1608.07187)) show that even word embeddings trained on Google News articles exhibit female/male gender stereotypes to a disturbing extent. This raises concerns because of their widespread use, as we describe, often tends to amplify these biases. 

In the following, we provide step-by-step instructions to demonstrate and quanitfy the biases in word embedding.



In [1]:
# Setup:
# Clone the code repository from https://github.com/tolga-b/debiaswe.git
# mkdir debiaswe_tutorial
# cd debiaswe_tutorial
# git clone https://github.com/tolga-b/debiaswe.git

# To reduce the time of downloading data, we provide as subset of GoogleNews-vectors in the following location:
# https://drive.google.com/file/d/1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG/view?usp=sharing

# For full embeddings:
# Download embeddings at https://github.com/tolga-b/debiaswe and put them on the following directory
# embeddings/GoogleNews-vectors-negative300-hard-debiased.bin
# embeddings/GoogleNews-vectors-negative300.bin

In [2]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions

## Part 1: Gender Bias in Word Embedding


### Step 1: Load data
We first load the word embedding trained on a corpus of Google News texts consisting of 3 million English words and terms. The embedding maps each word into a 300-dimension vector. 

In [3]:
# load google news word2vec
E = WordEmbedding('./embeddings/w2v_gnews_small.txt')

# load professions
professions = load_professions()
profession_words = [p[0] for p in professions]

*** Reading data from ./embeddings/w2v_gnews_small.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


### Step 2: Define gender direction

We define gender direction by the direciton of she - he because they are frequent and do not have fewer alternative word senses (e.g., man can also refer to mankind). In the paper, we discuss alternative approach for defining gender direction (e.g., using PCA).

In [4]:
# gender direction
v_gender = E.diff('she', 'he')

### Step 3: Generating analogies of "Man: x :: Woman : y"

We show that the word embedding model generates gender-streotypical analogy pairs. 
To generate the analogy pairs, we use the analogy score defined in our paper. This score finds word pairs that are well aligned with gender direction as well as within a short distance from each other to preserve topic consistency. 


In [5]:
# analogies gender
a_gender = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender:
    print(a+"-"+b)

Computing neighbors
Mean: 10.219732808538016
Median: 7.0
she-he
herself-himself
her-his
woman-man
daughter-son
businesswoman-businessman
girl-boy
actress-actor
chairwoman-chairman
heroine-hero
mother-father
spokeswoman-spokesman
sister-brother
girls-boys
sisters-brothers
queen-king
niece-nephew
councilwoman-councilman
motherhood-fatherhood
women-men
petite-lanky
ovarian_cancer-prostate_cancer
Anne-John
schoolgirl-schoolboy
granddaughter-grandson
aunt-uncle
matriarch-patriarch
twin_sister-twin_brother
mom-dad
lesbian-gay
husband-younger_brother
gal-dude
lady-gentleman
sorority-fraternity
mothers-fathers
grandmother-grandfather
blouse-shirt
soprano-baritone
queens-kings
Jill-Greg
daughters-sons
grandma-grandpa
volleyball-football
diva-superstar
mommy-kid
Sarah-Matthew
hairdresser-barber
softball-baseball
goddess-god
Aisha-Jamal
waitress-waiter
princess-prince
filly-colt
mare-gelding
ladies-gentlemen
childhood-boyhood
interior_designer-architect
nun-priest
wig-beard
granddaughters-grandso

### Step 4: Analyzing gender bias in word vectors asscoiated with professions

Next, we show that many occupations are unintendedly associated with either male of female by projecting their word vectors onto the gender dimension. 

The script will output the profession words sorted with respect to the projection score in the direction of gender.

In [6]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.23798442, 'maestro'),
  (-0.21665451, 'statesman'),
  (-0.20758669, 'skipper'),
  (-0.20267202, 'protege'),
  (-0.2020676, 'businessman'),
  (-0.19492392, 'sportsman'),
  (-0.18836352, 'philosopher'),
  (-0.1807366, 'marksman'),
  (-0.1728986, 'captain'),
  (-0.16785555, 'architect'),
  (-0.16702037, 'financier'),
  (-0.16313636, 'warrior'),
  (-0.15280862, 'major_leaguer'),
  (-0.15001445, 'trumpeter'),
  (-0.14718868, 'broadcaster'),
  (-0.14637242, 'magician'),
  (-0.14401694, 'fighter_pilot'),
  (-0.13782285, 'boss'),
  (-0.137182, 'industrialist'),
  (-0.13684885, 'pundit')],
 [(0.19714224, 'interior_designer'),
  (0.20833439, 'housekeeper'),
  (0.21560375, 'stylist'),
  (0.2236317, 'bookkeeper'),
  (0.23776126, 'maid'),
  (0.24125955, 'nun'),
  (0.24782579, 'nanny'),
  (0.24929334, 'hairdresser'),
  (0.24946158, 'paralegal'),
  (0.25276464, 'ballerina'),
  (0.25718823, 'socialite'),
  (0.26647124, 'librarian'),
  (0.27317622, 'receptionist'),
  (0.27540293, 'waitress'),
  (

## Part 2 Racial Bias

### Step 5: Define racial direction
We define racial direction based on the common names in different Demographic groups. 

In [7]:
names = ["Emily", "Aisha", "Anne", "Keisha", "Jill", "Tamika", "Allison", "Lakisha", "Laurie", "Tanisha", "Sarah",
         "Latoya", "Meredith", "Kenya", "Carrie", "Latonya", "Kristen", "Ebony", "Todd", "Rasheed", "Neil", "Tremayne",
         "Geoffrey", "Kareem", "Brett", "Darnell", "Brendan", "Tyrone", "Greg", "Hakim", "Matthew", "Jamal", "Jay",
         "Leroy", "Brad", "Jermaine"]
names_group1 = [names[2 * i] for i in range(len(names) // 2)]
names_group2 = [names[2 * i + 1] for i in range(len(names) // 2)]

In [8]:
# racial direction
vs = [sum(E.v(w) for w in names) for names in (names_group2, names_group1)]
vs = [v / np.linalg.norm(v) for v in vs]

v_racial = vs[1] - vs[0]
v_racial = v_racial / np.linalg.norm(v_racial)

In [26]:
names_group1

['Emily',
 'Anne',
 'Jill',
 'Allison',
 'Laurie',
 'Sarah',
 'Meredith',
 'Carrie',
 'Kristen',
 'Todd',
 'Neil',
 'Geoffrey',
 'Brett',
 'Brendan',
 'Greg',
 'Matthew',
 'Jay',
 'Brad']

### Step 6: Generating racial biased analogies

Similar to Step 3, we generate analogies that align with the racial dimension. 

In [9]:
# racial analogies
a_racial = E.best_analogies_dist_thresh(v_racial)

for (a,b,c) in a_racial:
    print(a+"-"+b)

Sarah-Keisha
defensemen-cornerbacks
hipster-hip_hop
punter-cornerback
singer_songwriter-rapper
defenseman-defensive_tackle
pole_vault-triple_jump
musicians-artistes
musician-artiste
catcher-wide_receiver
rock_n_roll-reggae
kicker-kick_returner
tavern-barbershop
freestyle_relay-meter_hurdles
lefthander-swingman
bacon-fried_chicken
artists-rappers
equipment-equipments
hockey-basketball
wool-cotton
unassisted_goal-layup
chocolates-sweets
buddy-cousin
priest-preacher
blue-black
medley_relay-meter_dash
quirky-funky
rabbi-imam
grapes-mango
telecommunications-telecommunication
pitchers-defensive_linemen
passages-verses
er-o
acoustic-soulful
punting-punt_returns
thefts-armed_robbery
bar-nightclub
digs-rebounds
Greg-Geoffrey
cellist-saxophonist
smarts-quickness
puck-halfcourt
quarterback-tailback
fox-leopard
pedophiles-rapists
potatoes-flour
en-el
infrastructure-infrastructural
evangelism-gospel
fiance-aunt
pointers-dunks
baseman-defensive_lineman
pedophile-rapist
joked-smiled
beer-soft_drink
g

In [27]:
a_racial

[('Sarah', 'Keisha', 0.6705212),
 ('defensemen', 'cornerbacks', 0.37196758),
 ('hipster', 'hip_hop', 0.3597801),
 ('punter', 'cornerback', 0.35276967),
 ('singer_songwriter', 'rapper', 0.343185),
 ('defenseman', 'defensive_tackle', 0.34279612),
 ('pole_vault', 'triple_jump', 0.33900595),
 ('musicians', 'artistes', 0.328106),
 ('musician', 'artiste', 0.32377928),
 ('catcher', 'wide_receiver', 0.3119021),
 ('rock_n_roll', 'reggae', 0.30865395),
 ('kicker', 'kick_returner', 0.30851004),
 ('tavern', 'barbershop', 0.30634567),
 ('freestyle_relay', 'meter_hurdles', 0.30112305),
 ('lefthander', 'swingman', 0.29952365),
 ('bacon', 'fried_chicken', 0.29813218),
 ('artists', 'rappers', 0.29516053),
 ('equipment', 'equipments', 0.29461268),
 ('hockey', 'basketball', 0.2851208),
 ('wool', 'cotton', 0.28036016),
 ('unassisted_goal', 'layup', 0.2802736),
 ('chocolates', 'sweets', 0.28023076),
 ('buddy', 'cousin', 0.27301168),
 ('priest', 'preacher', 0.2727967),
 ('blue', 'black', 0.2698871),
 ('medl

### Step 7: Analyzing racial bias in word vectors asscoiated with professions

Similar to Step 4, we project occpurations onto the racial dimension.

In [10]:
# profession analysis racial
sp = sorted([(E.v(w).dot(v_racial), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.3154624, 'artiste'),
  (-0.2736962, 'shopkeeper'),
  (-0.27285585, 'taxi_driver'),
  (-0.2424875, 'cab_driver'),
  (-0.230962, 'preacher'),
  (-0.21709056, 'boxer'),
  (-0.20973532, 'laborer'),
  (-0.20361683, 'barber'),
  (-0.1962502, 'cleric'),
  (-0.18273097, 'bodyguard'),
  (-0.18250427, 'gangster'),
  (-0.18162958, 'singer'),
  (-0.1687708, 'maid'),
  (-0.16871038, 'entertainer'),
  (-0.16197535, 'cabbie'),
  (-0.15332885, 'housewife'),
  (-0.14839591, 'civil_servant'),
  (-0.14115772, 'policeman'),
  (-0.13648951, 'minister'),
  (-0.13296556, 'drug_addict')],
 [(0.08779256, 'organist'),
  (0.09007451, 'philanthropist'),
  (0.09135294, 'cinematographer'),
  (0.09318026, 'manager'),
  (0.09358391, 'investment_banker'),
  (0.09687842, 'professor_emeritus'),
  (0.097828984, 'curator'),
  (0.09864862, 'freelance_writer'),
  (0.099171564, 'programmer'),
  (0.10142015, 'screenwriter'),
  (0.10198847, 'author'),
  (0.10438656, 'inventor'),
  (0.1067784, 'adventurer'),
  (0.10964737

## Exercise

Repeat Step 2-4 with debiased word embedding. 

You can use debiaswe debias function to do the debiasing with word sets of your choosing

You can leave equalize_pairs and gender_specific_words blank when coming up with your own groups. We give an example for the case of gender below for you to warm up.

In [19]:
from debiaswe.debias import debias

In [20]:
# Lets load some gender related word lists to help us with debiasing
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [['woman', 'man'], ['girl', 'boy'], ['she', 'he'], ['mother', 'father'], ['daughter', 'son'], ['gal', 'guy'], ['female', 'male'], ['her', 'his'], ['herself', 'himself'], ['Mary', 'John']]
gender specific 218 ['actress', 'actresses', 'aunt', 'aunts', 'bachelor', 'ballerina', 'barbershop', 'baritone', 'beard', 'beards']


In [21]:
debias(E, gender_specific_words, defs, equalize_pairs)

26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
{('PROSTATE_CANCER', 'OVARIAN_CANCER'), ('FELLA', 'GRANNY'), ('PRINCE', 'PRINCESS'), ('Males', 'Females'), ('SON', 'DAUGHTER'), ('CHAIRMAN', 'CHAIRWOMAN'), ('WIVES', 'HUSBANDS'), ('grandson', 'granddaughter'), ('GRANDFATHER', 'GRANDMOTHER'), ('TWIN_BROTHER', 'TWIN_SISTER'), ('gentlemen', 'ladies'), ('He', 'She'), ('SCHOOLBOY', 'SCHOOLGIRL'), ('son', 'daughter'), ('Male', 'Female'), ('spokesman', 'spokeswoman'), ('FATHERS', 'MOTHERS'), ('GRANDPA', 'GRANDMA'), ('kings', 'queens'), ('Father', 'Mother'), ('boys', 'girls'), ('grandpa', 'grandma'), ('Boy', 'Girl'), ('Gentleman', 'Lady'), ('Kings', 'Queens'), ('man', 'woman'), ('fatherhood', 'motherhood'), ('Colt', 'Filly'), ('king', 'queen'), ('Chairman', 'Chairwoman'), ('KINGS', 'QUEENS'), ('FRATERNITY', 'SORORITY'), ('Nephew', 'Niece'), ('MONASTERY', 'CONVENT'), ('Fella', 'Granny'), ('KING', 'QUEEN'), ('Schoolboy', 'Schoolgirl'), ('congressman', 'congresswoma

In [22]:
# profession analysis gender
sp_debiased = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp_debiased[0:20], sp_debiased[-20:]

([(-0.4196325, 'congressman'),
  (-0.4067585, 'businessman'),
  (-0.32398775, 'councilman'),
  (-0.30967075, 'dad'),
  (-0.21665451, 'statesman'),
  (-0.11345412, 'salesman'),
  (-0.073004864, 'monk'),
  (-0.072163954, 'handyman'),
  (-0.04946825, 'minister'),
  (-0.043583874, 'archbishop'),
  (-0.040207226, 'bishop'),
  (-0.03833246, 'commissioner'),
  (-0.03572439, 'surgeon'),
  (-0.033134006, 'trader'),
  (-0.032377213, 'observer'),
  (-0.032095864, 'neurosurgeon'),
  (-0.031450115, 'priest'),
  (-0.031133952, 'skipper'),
  (-0.029659169, 'lawmaker'),
  (-0.029511213, 'commander')],
 [(0.029965645, 'teenager'),
  (0.030237027, 'instructor'),
  (0.03094614, 'student'),
  (0.031116948, 'paralegal'),
  (0.032039408, 'bookkeeper'),
  (0.032434624, 'cinematographer'),
  (0.034329075, 'graphic_designer'),
  (0.034705658, 'lifeguard'),
  (0.03566652, 'janitor'),
  (0.03597192, 'drummer'),
  (0.042120155, 'wrestler'),
  (0.043902237, 'hairdresser'),
  (0.04813315, 'firefighter'),
  (0.23776

In [23]:
# analogies gender
a_gender_debiased = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender_debiased:
    print(a+"-"+b)

Computing neighbors
Mean: 10.218597434053665
Median: 7.0
spokeswoman-spokesman
filly-colt
mothers-fathers
princess-prince
women-men
girls-boys
grandmother-grandfather
granddaughters-grandsons
estrogen-testosterone
queens-kings
daughter-son
daughters-sons
woman-man
queen-king
chairwoman-chairman
moms-dads
girl-boy
niece-nephew
motherhood-fatherhood
congresswoman-congressman
sisters-brothers
females-males
councilwoman-councilman
she-he
ladies-gentlemen
herself-himself
her-his
aunt-uncle
schoolgirl-schoolboy
businesswoman-businessman
sorority-fraternity
twin_sister-twin_brother
female-male
ovarian_cancer-prostate_cancer
grandma-grandpa
ex_boyfriend-ex_girlfriend
husbands-wives
mare-gelding
mom-dad
convent-monastery
mother-father
sister-brother
gals-dudes
granddaughter-grandson
actress-actor
lesbian-gay
compatriot-countryman
husband-younger_brother
gal-dude
hers-theirs
heroine-protagonist
feminism-feminist
actresses-actors
childhood-boyhood
waitress-waiter
kid-guy
me-him
mommy-daddy
aunts-